In [ ]:
import requests
from requests.auth import HTTPBasicAuth
import json
import time
import itertools
import numpy as np
import pandas as pd

In [ ]:
def foo(page):
    return requests.get("https://api.github.com/repos/rust-lang/rust/issues", params={
        "state": "all",
        "filter": "all",
        "per_page": 100,
        "page": page
    }, auth=HTTPBasicAuth("user", "passowrd"))

In [ ]:
docs = []

In [ ]:
i = 258
while True:
    request = foo(i)
    if request.ok:
        docs.append(request.text)
        print(i, "Sucessful", len(request.text))
    else:
        print(i, "Failed")
        break
    i += 1

In [ ]:
docs = [json.loads(x) for x in set(docs)]

In [ ]:
issues = []
for x in docs:
    issues += x

In [ ]:
labels = []
for issue in issues:
    labels.append([x['name'] for x in issue['labels']])

In [ ]:
pairs = []
for x in labels:
    pairs += [tuple(sorted(list(y))) for y in itertools.combinations_with_replacement(x, 2)]
pairs.sort()

In [ ]:
counts = dict((key, sum(1 for _ in group)) for key, group in itertools.groupby(pairs))

In [ ]:
sims = dict((key, count / (counts[(key[0], key[0])] + counts[(key[1], key[1])] - count)) for key, count in counts.items()) 

In [ ]:
df = pd.DataFrame({
    "first_label": [x[0] for x in sims],
    "second_label": [x[1] for x in sims],
    "sim": [x for x in sims.values()]
})

In [ ]:
df = df.query("first_label != second_label")

In [ ]:
df.to_csv("../data/sims.csv", index=False)

In [ ]:
df_triage = pd.DataFrame(np.union1d(df["first_label"], df["second_label"]), columns=["label"])

In [ ]:
df_triage["triage"] = df_triage["label"].str.split("-", 1).map(lambda x: x[0])

In [ ]:
df_triage["triage"][df_triage["triage"].str.len() > 2] = ""

In [ ]:
df_triage.to_csv("../data/traige.csv", index=False)